# Layers

> Potentially helpful layers for your models

In [ ]:
#| default_exp layers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch, numpy as np, torch.nn.functional as F

from typing import Optional
from torch import nn
from torch import Tensor
from torch.fft import fft
from pftsleep.augmentations import create_patch, mask_patches_simple, jitter_augmentation, shuffle_dim, reverse_sequence
from pftsleep.signal import stft
from torch.nn.attention import SDPBackend, sdpa_kernel

## Linear Layers for Patches

In [ ]:
#| export
class PatchEncoder(nn.Module):
    def __init__(self, 
                 c_in, # the number of input channels
                 patch_len, # the length of the patches (either stft or interval length)
                 d_model, # the dimension of the initial linear layers for inputting patches into transformer
                 shared_embedding, # indicator of whether to project each channel individually or together
                 ):
        super().__init__()

        self.shared_embedding = shared_embedding
        self.n_vars = c_in

        # Input encoding: projection of feature vectors onto a d-dim vector space
        ## note that this could be an MLP too, if you want
        if not shared_embedding:
            self.W_P = nn.ModuleList()
            for _ in range(self.n_vars): self.W_P.append(nn.Linear(patch_len, d_model))
        else:
            self.W_P = nn.Linear(patch_len, d_model)

    def forward(self, x) -> Tensor:          
        """
        input: x: tensor [bs x num_patch x nvars x patch_len]
        returns: x: tensor [bs x num_patch x nvars x d_model]
        """
        # Input embedding
        if not self.shared_embedding:
            x_out = []
            for i in range(self.n_vars): 
                z = self.W_P[i](x[:,:,i,:])
                x_out.append(z)
            x = torch.stack(x_out, dim=2)
        else:
            x = self.W_P(x) # x: [bs x num_patch x nvars x d_model]
        return x

## Positional Encoding Layers

In [ ]:
#| export
class PositionalEncoding(nn.Module):
    def __init__(self, 
                 num_patch, # number of patches of time series or stft in input
                 d_model, # dimension of patch embeddings
                 #dropout=0.1 # dropout value
                 ):
        super().__init__()
        self.num_patch = num_patch
        self.d_model = d_model
        
        # Positional encoding - learned
        self.W_pos =  nn.Parameter(torch.empty((num_patch, d_model)))
        nn.init.uniform_(self.W_pos, -0.02, 0.02)
        #self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        """
        input: x: [bs * nvars x num_patch x d_model]
        returns: x: [bs * nvars x num_patch x d_model]
        """
        x = x + self.W_pos
        return x


In [ ]:
#| export
class tAPE(nn.Module):
    """
    time Absolute Position Encoding
    Adapted from tsai
    """

    def __init__(self, 
        d_model:int, # the embedding dimension
        seq_len:int, # the max. length of the incoming sequence or num patches
        #dropout:float=0., # dropout value
        scale_factor=1.0
        ):
        super().__init__()
        
        pe = torch.zeros(seq_len, d_model)  # positional encoding
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin((position * div_term)*(d_model/seq_len)) # this is the difference between normal PE and tAPE, scaling (d_model/seq_len)
        pe[:, 1::2] = torch.cos((position * div_term)*(d_model/seq_len))
        pe = scale_factor * pe.unsqueeze(0)
        self.register_buffer('pe', pe)  # this stores the variable in the state_dict (used for non-trainable variables)

        #self.dropout = nn.Dropout(p=dropout)

    def forward(self, x): # [batch size, sequence length, embed dim]
        x = x + self.pe
        return x

## Mask and Augmentation Layers

In [ ]:
#| export
class Mask(nn.Module):
    def __init__(self, mask_type, mask_ratio, return_mask=True):
        super().__init__()
        assert mask_type in ['patch', 'jitter_zero']
        self.mask_type = mask_type
        self.mask_ratio = mask_ratio
        self.return_mask = return_mask
    
    def forward(self, x):
        if self.mask_type == 'jitter_zero':
            # mask is a number with the number of masks applied in this function
            x_masked, mask = jitter_augmentation(x, mask_ratio=(self.mask_ratio/2), jitter_ratio=(self.mask_ratio/2))
        else:
            # padding mask is currently not implemented here.. tbh not sure if its needed
            x_masked, mask = mask_patches_simple(x, mask_ratio=self.mask_ratio)
        if self.return_mask:
            return x_masked, mask
        else:
            return x_masked


In [ ]:
#| notest
torch.manual_seed(125)
m = Mask(mask_type='jitter_zero', mask_ratio=0.5)
x = torch.randn((9))

m(x), m(x)

((tensor([ 0.0000,  0.0000,  0.0000,  1.0975,  0.0000,  1.4248,  0.0000, -0.1104,
           1.1865]),
  tensor(8)),
 (tensor([ 1.1462, -1.8379, -0.2368,  2.0488,  0.0000,  0.0000, -0.3927, -1.0523,
           0.1294]),
  tensor(4)))

In [ ]:
#| notest
x = torch.randn((4))
x_aug = x.clone()
mask = torch.tensor([True,True,False,False])

x_aug[mask] = 1
x_aug, x

(tensor([ 1.0000,  1.0000,  1.9390, -0.0338]),
 tensor([-1.7385, -0.5780,  1.9390, -0.0338]))

In [ ]:
#| export
class PatchAugmentations(nn.Module):
    def __init__(self, augmentations=['patch_mask', 'jitter_zero_mask', 'reverse_sequence', 'shuffle_channels'], patch_mask_ratio=0., jitter_zero_mask_ratio=0.):
        super().__init__()
        if 'patch_mask' in augmentations:
            assert patch_mask_ratio >= 0.
        if 'jitter_zero_mask' in augmentations:
            assert jitter_zero_mask_ratio >= 0.
        self.augmentations = augmentations
        self.patch_mask_ratio = patch_mask_ratio
        self.jitter_zero_mask_ratio = jitter_zero_mask_ratio
    def forward(self, x):
        if 'patch_mask' in self.augmentations and 'jitter_zero_mask' in self.augmentations:
            mask_choice = np.random.choice(['patch_mask', 'jitter_zero_mask'], replace=True)
        else:
            mask_choice = None
        augs = self.augmentations.copy()
        np.random.shuffle(augs)
        for augmentation in augs:
            if augmentation == 'jitter_zero_mask' and (mask_choice == 'jitter_zero_mask' or mask_choice is None):
                # mask is a number with the number of masks applied in this function
                x, _ = jitter_augmentation(x, mask_ratio=(self.jitter_zero_mask_ratio/2), jitter_ratio=(self.jitter_zero_mask_ratio/2))
            if augmentation == 'patch_mask' and (mask_choice == 'patch_mask' or mask_choice is None):
                # padding mask is currently not implemented here.. tbh not sure if its needed
                x, _ = mask_patches_simple(x, mask_ratio=self.patch_mask_ratio)
            if augmentation == 'shuffle_channels':
                x = shuffle_dim(x, dim=2, p=0.5)
            if augmentation == 'reverse_sequence':
                x = reverse_sequence(x, seq_dim=(-1,), p=0.5)
        return x

In [ ]:
#| notest
x = torch.randn(4,3600,7,750)

s=PatchAugmentations(patch_mask_ratio=0.1, jitter_zero_mask_ratio=0.1)
s(x).shape

torch.Size([4, 3600, 7, 750])

In [ ]:
#| export
class EmbeddingAugmentations(nn.Module):
    def __init__(self, augmentations=['shuffle_dims', 'jitter_zero_mask', 'patch_mask'], dims_to_shuffle = [1,2,3], patch_mask_ratio=0., jitter_zero_mask_ratio=0.):
        super().__init__()
        assert set(augmentations) - set(['shuffle_dims', 'jitter_zero_mask', 'patch_mask']) == set(), f"Augmentations must be in {set(['shuffle_dims', 'jitter_zero_mask', 'patch_mask'])}."
        if 'patch_mask' in augmentations:
            assert patch_mask_ratio >= 0.
        if 'jitter_zero_mask' in augmentations:
            assert jitter_zero_mask_ratio >= 0.
        self.augmentations = augmentations
        self.patch_mask_ratio = patch_mask_ratio
        self.jitter_zero_mask_ratio = jitter_zero_mask_ratio
        self.dims_to_shuffle = dims_to_shuffle
    
    def forward(self, x):
        """
        Input is an embedding: 
        x: [bs x n channels x d model x n patches]
        dims correspond to 1 = channels 2 = n patches 3 = d model
        returns: [bs x n channels x d model x n patches]
        """
        x = x.permute(0,1,3,2)
        if 'patch_mask' in self.augmentations and 'jitter_zero_mask' in self.augmentations:
            mask_choice = np.random.choice(['patch_mask', 'jitter_zero_mask'], replace=True)
        else:
            mask_choice = None
        augs = self.augmentations
        np.random.shuffle(augs)
        for augmentation in augs:
            if augmentation == 'jitter_zero_mask' and (mask_choice == 'jitter_zero_mask' or mask_choice is None):
                # mask is a number with the number of masks applied in this function
                x, _ = jitter_augmentation(x, mask_ratio=(self.jitter_zero_mask_ratio/2), jitter_ratio=(self.jitter_zero_mask_ratio/2))
            if augmentation == 'patch_mask' and (mask_choice == 'patch_mask' or mask_choice is None):
                # padding mask is currently not implemented here.. tbh not sure if its needed
                x, _ = mask_patches_simple(x, mask_ratio=self.patch_mask_ratio)
            if augmentation == 'shuffle_dims':
                # shuffle channels, patches, features
                dim_shuffle_order = self.dims_to_shuffle
                np.random.shuffle(dim_shuffle_order)
                for dim in dim_shuffle_order:
                    x = shuffle_dim(x, dim=dim, p=0.5)
        x = x.permute(0,1,3,2)
        return x

In [ ]:
#| notest
x = torch.randn(4,7,512,3600)

s = EmbeddingAugmentations(augmentations=['jitter_zero_mask'], dims_to_shuffle=[1], patch_mask_ratio=0.1, jitter_zero_mask_ratio=0.1)
s(x).shape

torch.Size([4, 7, 512, 3600])

## Patch and Fourier Layers

In [ ]:
#| export
class Patch(nn.Module):
    def __init__(self, patch_len, stride, max_seq_len=None):
        super().__init__()
        self.patch_len = patch_len
        self.stride = stride
        self.max_seq_len = max_seq_len
    
    def forward(self, x, constant_pad=False, constant_pad_value=0):
        x = create_patch(x, patch_len=self.patch_len, stride=self.stride, constant_pad=constant_pad, constant_pad_value=constant_pad_value, max_seq_len=self.max_seq_len)
        return x

In [ ]:
#| export 
class STFT(nn.Module):
    def __init__(self, 
                 n_fft, 
                 win_length,
                 hop_length, 
                 stft_norm, 
                 decibel_scale, 
                 channel_stft_means=None, 
                 channel_stft_stds=None, 
                 pad_win_length_to_nfft=True, 
                 pad_mode='reflect', 
                 center=False, 
                 return_complex=True
                 ):
        super().__init__()
        self.n_fft = n_fft
        self.win_length = win_length
        self.hop_length = hop_length
        self.pad_mode = pad_mode
        self.pad_win_length_to_nfft = pad_win_length_to_nfft
        self.center = center
        self.return_complex = return_complex
        self.stft_norm = stft_norm
        self.decibel_scale = decibel_scale
        self.channel_stft_means = channel_stft_means
        self.channel_stft_stds = channel_stft_stds

    def forward(self, x):
        """
        x: [bs x n_vars x max_seq_len]
        out: [bs x n_vars x n_fft // 2 + 1 x stft_len]
        """
        x_fft = stft(x, 
                     n_fft=self.n_fft, 
                     win_length=self.win_length,
                     normalized=self.stft_norm,
                     pad_win_length_to_nfft=self.pad_win_length_to_nfft,
                     decibel_scale=self.decibel_scale, 
                     channel_stft_means=self.channel_stft_means, 
                     channel_stft_stds=self.channel_stft_stds, 
                     hop_length=self.hop_length, 
                     pad_mode=self.pad_mode, 
                     center=self.center, 
                     return_complex=self.return_complex
                     )
        return x_fft


In [ ]:
#| export 
class FFT(nn.Module):
    def __init__(self, 
                 dim=-1, # dimension to calculate fft over
                 norm='backward' #  "forward" - normalize by 1/n, "backward" - no normalization, "ortho" - normalize by 1/sqrt(n) (making the FFT orthonormal)
                 ):
        super().__init__()
        self.dim = dim
        self.norm = norm

    def forward(self, x):
        """
        This is equivalent to an stft with onesided set to False, hop length of 1, and win_length == n_fft.

        x: [bs x num_patch x n_vars x patch_len]
        out: [bs x num_patch x n_vars x patch_len(freq domain)]
        """
        x_fft = fft(x, dim=self.dim, norm=self.norm).abs()
        return x_fft


## Reversible Instance Normalization

In [ ]:
#| export
class RevIN(nn.Module):
    def __init__(self, 
                 num_features: int, # the number of channels or features in the input
                 eps=1e-5, # added to avoid division by zero errors
                 dim_to_reduce=-1, # the dimension to reduce, 
                 affine=True # learning affine parameters bias and weight per channel
                 ):
        """
        
        """
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.dim_to_reduce = dim_to_reduce

        if self.affine:
            self.affine_weight = nn.Parameter(torch.ones(1,num_features,1))
            self.affine_bias = nn.Parameter(torch.zeros(1,num_features,1))

    def forward(self, x, mode:bool):
        """
        x: [bs x n_vars x max_seq_len]
        """
        if mode:
            return self._normalize(x)
        else:
            return self._denormalize(x)

    def _normalize(self, x):      
        self.mean = torch.mean(x, dim=self.dim_to_reduce, keepdim=True).detach()
        self.stdev = torch.std(x, dim=self.dim_to_reduce, keepdim=True, unbiased=False).detach() + self.eps

        x = x.sub(self.mean)
        x = x.div(self.stdev)
        
        if self.affine:
            x = x.mul(self.affine_weight)
            x = x.add(self.affine_bias)
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x.sub(self.affine_bias)
            x = x.div(self.affine_weight)# + self.eps*self.eps)
        x = x.mul(self.stdev)
        x = x.add(self.mean)
        return x

In [ ]:
#| notest
x = torch.randn(4,7,1000)

revin = RevIN(x.shape[1], dim_to_reduce=-1, affine=True)
x_norm = revin(x, mode=True)
x_denorm = revin(x_norm, mode=False)

x.shape, x_norm.shape, x_denorm.shape, revin.mean.shape, revin.stdev.shape

(torch.Size([4, 7, 1000]),
 torch.Size([4, 7, 1000]),
 torch.Size([4, 7, 1000]),
 torch.Size([4, 7, 1]),
 torch.Size([4, 7, 1]))

## Attention

In [ ]:
#| export
class MultiheadFlashAttention(nn.Module):
    """Multihead attention layer with optional causal masking.
    Uses flash attention when available in PyTorch 2.0+.
    
    Args:
        n_heads (int): Number of attention heads
        d_model (int): Embedding dimension
        qkv_bias (bool, optional): Use bias in linear layers. Defaults to False.
        is_causal (bool, optional): Use causal masking. Defaults to False.
        attn_dropout (float, optional): Attention dropout probability. Defaults to 0.0.
        proj_dropout (float, optional): Dropout probability. Defaults to 0.0.

    Note that when passing in a key paddings mask, it should be a boolean tensor of shape [bs x seq_len]
    where a True value indicates that the key at that position should be ignored for the purposes of attention.
    This is contrary to what the PyTorch documentation suggests, but is correct in this module.
    """, 
    def __init__(self, d_model: int, n_heads: int, qkv_bias: bool=True, 
                 is_causal: bool=False, attn_dropout: float=0.0, proj_dropout: float=0.0):
        super().__init__()
        assert d_model % n_heads == 0
        
        # # key, query, value projections for all heads, but in a batch, Combined Q,K,V projections
        self.c_attn = nn.Linear(d_model, 3 * d_model, bias=qkv_bias)
        # Output projection
        self.c_proj = nn.Linear(d_model, d_model, bias=qkv_bias)
        head_dim = d_model // n_heads
        self.scale = head_dim ** -0.5
        
        # Regularization
        self.attn_dropout = attn_dropout
        self.resid_dropout = nn.Dropout(proj_dropout)
        
        # Architecture
        self.num_heads = n_heads
        self.embed_dimension = d_model
        self.is_causal = is_causal

    def forward(self, x: Tensor, 
                key_padding_mask: Optional[Tensor] = None
                ) -> Tensor:
        """
        x: [bs x seq_len x d_model]
        key_padding_mask: [bs x seq_len]
        """
        batch_size = x.size(0)
        
        # Project to Q,K,V
        qkv = self.c_attn(x)
        embed_dim = qkv.size(2)
        head_dim = embed_dim // (self.num_heads * 3)

        # Split and reshape
        query, key, value = qkv.chunk(3, -1)
        query = query.view(batch_size, -1, self.num_heads, head_dim).transpose(1, 2)
        key = key.view(batch_size, -1, self.num_heads, head_dim).transpose(1, 2)
        value = value.view(batch_size, -1, self.num_heads, head_dim).transpose(1, 2)

        # Set dropout and causality based on training mode
        attn_dropout = self.attn_dropout if self.training else 0.0
        is_causal = self.is_causal if self.training else False
        if key_padding_mask is not None:
            if key_padding_mask.dim() == 2:
                key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(1)  # [bs x 1 x 1 x seq_len]
            if key_padding_mask.dtype == torch.bool:
                key_padding_mask = key_padding_mask.float().masked_fill(key_padding_mask, float('-inf'))
            key_padding_mask = key_padding_mask.expand(-1, self.num_heads, query.size(-2), -1)
            if key_padding_mask.stride(-1) != 1:
                # see https://github.com/pytorch/pytorch/issues/127523
                key_padding_mask = torch.empty_like(key_padding_mask, memory_format=torch.contiguous_format).copy_(key_padding_mask)

        # Attention (with flash attention when available)
        with sdpa_kernel(SDPBackend.FLASH_ATTENTION):
            y = F.scaled_dot_product_attention(
                query, key, value, 
                attn_mask=key_padding_mask,  # PyTorch handles mask preprocessing internally, can pass attn_mask or key_padding_mask
                dropout_p=attn_dropout, 
                is_causal=is_causal,
                scale=self.scale
            )
        
        # Reshape and project output
        y = y.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * head_dim)
        y = self.resid_dropout(self.c_proj(y))
        
        return y

In [ ]:
#| notest
mha = MultiheadFlashAttention(d_model=512, n_heads=8, attn_dropout=0., proj_dropout=0.)
x = torch.randn(2*7,100,512) # [bs * n_vars x n_patches x d_model]
key_padding_mask = torch.zeros(2*7, 100, dtype=torch.bool)
key_padding_mask[:, -2:] = True  # mask last 2 positions
output = mha(x, key_padding_mask=key_padding_mask)
output.shape

torch.Size([14, 100, 512])

In [ ]:
#| export
class ScaledDotProductAttention(nn.Module):
    """Scaled Dot-Product Attention module (Attention is all you need by Vaswani et al., 2017) with optional residual attention from previous layer
    (Realformer: Transformer likes residual attention by He et al, 2020) and locality self sttention (Vision Transformer for Small-Size Datasets
    by Lee et al, 2021)"""

    def __init__(self, d_model, n_heads, attn_dropout=0., res_attention=False, lsa=False):
        super().__init__()
        self.attn_dropout = nn.Dropout(attn_dropout)
        self.res_attention = res_attention
        head_dim = d_model // n_heads
        self.scale = nn.Parameter(torch.tensor(head_dim ** -0.5), requires_grad=lsa)
        self.lsa = lsa

    def forward(self, q:Tensor, k:Tensor, v:Tensor, prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
        '''
        Input shape:
            q               : [bs x n_heads x max_q_len x d_k]
            k               : [bs x n_heads x d_k x seq_len]
            v               : [bs x n_heads x seq_len x d_v]
            prev            : [bs x n_heads x q_len x seq_len]
            key_padding_mask: [bs x seq_len]
            attn_mask       : [1 x seq_len x seq_len]
        Output shape:
            output:  [bs x n_heads x q_len x d_v]
            attn   : [bs x n_heads x q_len x seq_len]
            scores : [bs x n_heads x q_len x seq_len]
        '''

        # Scaled MatMul (q, k) - similarity scores for all pairs of positions in an input sequence
        ## this is the b ottleneck in your code
        attn_scores = torch.matmul(q, k) * self.scale      # attn_scores : [bs x n_heads x max_q_len x q_len]
        # Add pre-softmax attention scores from the previous layer (optional)
        if prev is not None: attn_scores = attn_scores + prev

        ## if implementing attention or key padding mask - https://github.com/pytorch/pytorch/issues/41508#issuecomment-1723119580
        # Attention mask (optional)
        if attn_mask is not None:                                     # attn_mask with shape [q_len x seq_len] - only used when q_len == seq_len
            if attn_mask.dtype == torch.bool:
                attn_scores.masked_fill_(attn_mask, -np.inf)
            else:
                attn_scores += attn_mask

        # Key padding mask (optional)
        if key_padding_mask is not None:# and torch.any(key_padding_mask): - not sure if this is needed                         # mask with shape [bs x q_len] (only when max_w_len == q_len)
            attn_scores.masked_fill_(key_padding_mask.unsqueeze(1).unsqueeze(2).to(torch.bool), -np.inf)

        # normalize the attention weights
        # this is another bottleneck (softmax)
        attn_weights = F.softmax(attn_scores, dim=-1)                 # attn_weights   : [bs x n_heads x max_q_len x q_len]
        if attn_mask is not None and attn_mask.dtype == torch.bool:
            attn_weights = attn_weights.masked_fill(attn_mask, 0.0)
       
        if key_padding_mask is not None:# and torch.any(key_padding_mask):
            attn_weights = attn_weights.masked_fill(key_padding_mask.unsqueeze(1).unsqueeze(2).to(torch.bool), 0.0)
       
        attn_weights = self.attn_dropout(attn_weights)
        # compute the new values given the attention weights
        output = torch.matmul(attn_weights, v)                        # output: [bs x n_heads x max_q_len x d_v]
        if self.res_attention: return output, attn_weights, attn_scores
        else: return output, attn_weights


In [ ]:
#| export
class MultiheadAttentionCustom(nn.Module):
    def __init__(self, d_model, n_heads, d_k=None, d_v=None, res_attention=False, attn_dropout=0., proj_dropout=0., qkv_bias=True, lsa=False):
        """Multi Head Attention Layer
        Input shape:
            Q:       [batch_size (bs) x max_q_len x d_model]
            K, V:    [batch_size (bs) x q_len x d_model]
            mask:    [q_len x q_len]
        """
        super().__init__()
        d_k = d_model // n_heads if d_k is None else d_k
        d_v = d_model // n_heads if d_v is None else d_v

        self.n_heads, self.d_k, self.d_v = n_heads, d_k, d_v

        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=qkv_bias)

        # Scaled Dot-Product Attention (multiple heads)
        self.res_attention = res_attention
        self.sdp_attn = ScaledDotProductAttention(d_model, n_heads, attn_dropout=attn_dropout, res_attention=self.res_attention, lsa=lsa)

        # Poject output
        self.to_out = nn.Sequential(nn.Linear(n_heads * d_v, d_model), nn.Dropout(proj_dropout))


    def forward(self, Q:Tensor, K:Optional[Tensor]=None, V:Optional[Tensor]=None, prev:Optional[Tensor]=None,
                key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):

        bs = Q.size(0)
        if K is None: K = Q
        if V is None: V = Q

        # Linear (+ split in multiple heads)
        q_s = self.W_Q(Q).view(bs, -1, self.n_heads, self.d_k).transpose(1,2)       # q_s    : [bs x n_heads x max_q_len x d_k]
        k_s = self.W_K(K).view(bs, -1, self.n_heads, self.d_k).permute(0,2,3,1)     # k_s    : [bs x n_heads x d_k x q_len] - transpose(1,2) + transpose(2,3)
        v_s = self.W_V(V).view(bs, -1, self.n_heads, self.d_v).transpose(1,2)       # v_s    : [bs x n_heads x q_len x d_v]
        # Apply Scaled Dot-Product Attention (multiple heads)
        if self.res_attention:
            output, attn_weights, attn_scores = self.sdp_attn(q_s, k_s, v_s, prev=prev, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        else:
            output, attn_weights = self.sdp_attn(q_s, k_s, v_s, key_padding_mask=key_padding_mask, attn_mask=attn_mask)
        # output: [bs x n_heads x q_len x d_v], attn: [bs x n_heads x q_len x q_len], scores: [bs x n_heads x max_q_len x q_len]

        # back to the original inputs dimensions
        output = output.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * self.d_v) # output: [bs x q_len x n_heads * d_v]
        output = self.to_out(output)

        if self.res_attention: return output, attn_weights, attn_scores
        else: return output, attn_weights

In [ ]:
#| notest
mha_attn = MultiheadAttentionCustom(d_model=512, n_heads=8, attn_dropout=0., proj_dropout=0., res_attention=False)
mha_attn

MultiheadAttentionCustom(
  (W_Q): Linear(in_features=512, out_features=512, bias=True)
  (W_K): Linear(in_features=512, out_features=512, bias=True)
  (W_V): Linear(in_features=512, out_features=512, bias=True)
  (sdp_attn): ScaledDotProductAttention(
    (attn_dropout): Dropout(p=0.0, inplace=False)
  )
  (to_out): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)

In [ ]:
#| notest
def test_attention_equivalence():
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Test parameters
    batch_size = 2
    seq_len = 10
    d_model = 64
    n_heads = 4
    
    # Create input tensor (only need one since we're using self-attention)
    x = torch.randn(batch_size, seq_len, d_model)
    
    # Create key padding mask
    key_padding_mask = torch.zeros(batch_size, seq_len, dtype=torch.bool)
    key_padding_mask[:, -2:] = True  # mask last 2 positions

    # Initialize both implementations
    custom_mha = MultiheadAttentionCustom(d_model=d_model, n_heads=n_heads)
    flash_mha = MultiheadFlashAttention(d_model=d_model, n_heads=n_heads)
    
    # Set both models to eval mode to disable dropout
    custom_mha.eval()
    flash_mha.eval()
    
    # Copy weights to ensure identical parameters
    # Combine QKV weights from custom implementation into single matrix for flash attention
    combined_weight = torch.cat([
        custom_mha.W_Q.weight,
        custom_mha.W_K.weight,
        custom_mha.W_V.weight
    ], dim=0)
    combined_bias = torch.cat([
        custom_mha.W_Q.bias,
        custom_mha.W_K.bias,
        custom_mha.W_V.bias
    ], dim=0)
    
    # Copy combined weights to flash attention
    flash_mha.c_attn.weight.data = combined_weight
    flash_mha.c_attn.bias.data = combined_bias
    
    # Output projection weights
    flash_mha.c_proj.weight.data = custom_mha.to_out[0].weight.data.clone()
    flash_mha.c_proj.bias.data = custom_mha.to_out[0].bias.data.clone()
    
    # Forward pass
    with torch.no_grad():
        custom_output, custom_attn = custom_mha(x, key_padding_mask=key_padding_mask)
        
        flash_output = flash_mha(x, attn_mask=key_padding_mask)
    
    # Compare outputs
    print(f"Custom output shape: {custom_output.shape}")
    print(f"Flash output shape: {flash_output.shape}")
    
    output_close = torch.allclose(custom_output, flash_output, rtol=0, atol=0)
    print(f"Outputs match: {output_close}")
    
    if not output_close:
        print("\nOutput differences:")
        print(f"Max difference: {(custom_output - flash_output).abs().max().item()}")
        print(f"Mean difference: {(custom_output - flash_output).abs().mean().item()}")
    
    return custom_output, flash_output

custom_output, flash_output = test_attention_equivalence()
#: 8.940696716308594e-08
#Mean difference: 1.0550138540565968e-08

Custom output shape: torch.Size([2, 10, 64])
Flash output shape: torch.Size([2, 10, 64])
Outputs match: True


In [ ]:
#| notest
d_model=512
n_heads=8
d_k = d_v = d_model // n_heads
attn = ScaledDotProductAttention(d_model=d_model, n_heads=n_heads)
mha_attn = MultiheadAttentionCustom(d_model, n_heads)

W_Q = nn.Linear(d_model, d_k * n_heads)
W_K = nn.Linear(d_model, d_k * n_heads)
W_V = nn.Linear(d_model, d_v * n_heads)
X,_,_ = ds[0]

X = create_patch(X, patch_len=(10*50), stride=(5*50), constant_pad=True)

patch_len = X.shape[-1]

X = X[None, ...].permute(0,2,1,3)  # simulate batch size of 1 [bs x n_vars x num_patch x patch_len]

print(f'X input shape: {X.shape}')
W_P = nn.Linear(patch_len, d_model)

X = W_P(X) # project to d_model
print(f"Projected X shape to d_model: {X.shape}")

X = torch.reshape(X, (X.shape[0]*X.shape[1],X.shape[2],X.shape[3]))
print(f"Reshape for attention: {X.shape}")

# test multihead attention
print("\nTesting MHA and SDA attention, with just 50 elements.")
mha_output, mha_attn_weights = mha_attn(Q=X[:,:50,:])
print(f"MHA attention output shape: {mha_output.shape}, mha attn weight shape: {mha_attn_weights.shape}")

# test scaled dot product attn
K = Q = V = X

# # Linear (+ split in multiple heads)
bs = 1 # 1 * 16
q_s = W_Q(Q).reshape(bs, -1, n_heads, d_k).transpose(1, 2)
k_s = W_K(K).reshape(bs, -1, n_heads, d_k).permute(0, 2, 3, 1)
v_s = W_V(V).reshape(bs, -1, n_heads, d_v).transpose(1, 2)
print(f"Q shape: {q_s.shape}, K shape: {k_s.shape}, V shape: {v_s.shape}")

to_out = nn.Linear(n_heads * d_v, d_model)
output, attn_weights = attn(q_s[:,:,:50,:],k_s[:,:,:,:50], v_s[:,:,:50,:])
output = output.transpose(1, 2).contiguous().view(bs, -1, n_heads * d_v)
print(f"Attn output shape {output.shape}, attn weight shape: {attn_weights.shape}")

X input shape: torch.Size([1, 7, 10799, 500])
Projected X shape to d_model: torch.Size([1, 7, 10799, 512])
Reshape for attention: torch.Size([7, 10799, 512])

Testing MHA and SDA attention, with just 50 elements.
MHA attention output shape: torch.Size([7, 50, 512]), mha attn weight shape: torch.Size([7, 8, 50, 50])
Q shape: torch.Size([1, 8, 75593, 64]), K shape: torch.Size([1, 8, 64, 75593]), V shape: torch.Size([1, 8, 75593, 64])
Attn output shape torch.Size([1, 50, 512]), attn weight shape: torch.Size([1, 8, 50, 50])


In [ ]:
#| export
class Attention_Rel_Scl(nn.Module):
    def __init__(self, 
        d_model:int, # Embedding dimension
        n_heads:int, # number of attention heads
        seq_len:int, # sequence length or num patches
        d_k:int=None, # key dimension
        d_v:int=None, # value dimension
        res_attention:bool=False, # whether to use residual attention
        attn_dropout:float=0., # dropout for attention
        lsa:bool=False, # whether to use LSA, trainable paramater for scaling
        proj_dropout:float=0., # dropout for projection
        qkv_bias:bool=True, # bias for q, k, v
        ):
        """
        Adapted from tsai
        Added residual attention and dropout
        """
        super().__init__()

        self.seq_len = seq_len
        head_dim = d_model // n_heads
        self.scale = nn.Parameter(torch.tensor(head_dim ** -0.5), requires_grad=lsa)

        d_k = d_model // n_heads if d_k is None else d_k
        d_v = d_model // n_heads if d_v is None else d_v

        self.n_heads, self.d_k, self.d_v = n_heads, d_k, d_v

        self.query = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.key = nn.Linear(d_model, d_k * n_heads, bias=qkv_bias)
        self.value = nn.Linear(d_model, d_v * n_heads, bias=qkv_bias)

        self.res_attention = res_attention

        self.relative_bias_table = nn.Parameter(torch.zeros((2 * self.seq_len - 1), n_heads))
        coords = torch.meshgrid((torch.arange(1), torch.arange(self.seq_len)), indexing="xy")
        coords = torch.flatten(torch.stack(coords), 1)
        relative_coords = coords[:, :, None] - coords[:, None, :]
        relative_coords[1] += self.seq_len - 1
        relative_coords = relative_coords.permute(1, 2, 0)
        relative_index = relative_coords.sum(-1).flatten().unsqueeze(1)
        self.register_buffer("relative_index", relative_index)

        self.attn_dropout = nn.Dropout(attn_dropout)
        self.to_out = nn.Sequential(nn.Linear(n_heads * d_v, d_model), nn.Dropout(proj_dropout))

    def forward(self, x, prev:Optional[Tensor]=None, key_padding_mask:Optional[Tensor]=None, attn_mask:Optional[Tensor]=None):
        batch_size = x.shape[0]
        
        q = self.query(x).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = self.key(x).view(batch_size, -1, self.n_heads, self.d_k).permute(0, 2, 3, 1)
        v = self.value(x).view(batch_size, -1, self.n_heads, self.d_v).transpose(1, 2)

        attn_scores = torch.matmul(q, k) * self.scale # [seq_len, seq_len]
        if prev is not None: attn_scores = attn_scores + prev

        if attn_mask is not None:                                     # attn_mask with shape [q_len x seq_len] - only used when q_len == seq_len
            if attn_mask.dtype == torch.bool:
                attn_scores.masked_fill_(attn_mask, -np.inf)
            else:
                attn_scores += attn_mask

        # Key padding mask (optional)
        if key_padding_mask is not None:# and torch.any(key_padding_mask): - not sure if this is needed                         # mask with shape [bs x q_len] (only when max_w_len == q_len)
            attn_scores.masked_fill_(key_padding_mask.unsqueeze(1).unsqueeze(2).to(torch.bool), -np.inf)
        
        attn_weights = F.softmax(attn_scores, dim=-1)

        relative_bias = self.relative_bias_table.gather(0, self.relative_index.repeat(1, self.n_heads))
        relative_bias = relative_bias.reshape(self.seq_len, self.seq_len, -1).permute(2, 0, 1).unsqueeze(0)
        attn_weights = attn_weights + relative_bias

        if attn_mask is not None and attn_mask.dtype == torch.bool:
            attn_weights = attn_weights.masked_fill(attn_mask, 0.0)
       
        if key_padding_mask is not None:# and torch.any(key_padding_mask):
            attn_weights = attn_weights.masked_fill(key_padding_mask.unsqueeze(1).unsqueeze(2).to(torch.bool), 0.0)

        attn_weights = self.attn_dropout(attn_weights)

        out = torch.matmul(attn_weights, v) # [batch_size, n_heads, seq_len, d_head]
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # output: [bs x q_len x n_heads * d_v]
        out = self.to_out(out)
        if self.res_attention: 
            return out, attn_weights, attn_scores
        else: 
            return out, attn_weights

In [ ]:
#| notest
d_model=16
c_in = 2
seq_len = 1*360*10
x = torch.randn(4,c_in,seq_len)
embed_layer = nn.Sequential(nn.Conv2d(1, d_model*4, kernel_size=[1, 7], padding='same'), nn.BatchNorm2d(d_model*4), nn.GELU())
embed_layer2 = nn.Sequential(nn.Conv2d(d_model*4, d_model, kernel_size=[c_in, 1], padding='valid'), nn.BatchNorm2d(d_model), nn.GELU())
abs_position = tAPE(d_model, seq_len=seq_len)
x_emb = embed_layer2(embed_layer(x.unsqueeze(1))).squeeze(2)
x_emb = x_emb.permute(0,2,1)
x_emb_pos = abs_position(x_emb)

model = Attention_Rel_Scl(d_model=d_model,
        n_heads=2, # number of attention heads
        seq_len=seq_len, # sequence length or num patches
        )

out, attn_weights = model(x_emb)

In [ ]:
#| notest
## test w patches [bs *c_in x num_patches x d_model]
d_model=512
c_in = 2
num_patches = 10
x_emb = torch.randn(4*c_in,num_patches, d_model)
abs_position = tAPE(d_model, seq_len=num_patches)
x_emb_pos = abs_position(x_emb)

model = Attention_Rel_Scl(d_model=d_model,
        n_heads=2, # number of attention heads
        seq_len=num_patches, # sequence length or num patches
        )

out, attn_weights = model(x_emb)


## Pretraining Heads

In [ ]:
#| export
class MaskedAutogressionFeedForward(nn.Module):
    def __init__(self, 
                 c_in, # the number of input channels
                 patch_len, # the length of the patches (either stft or interval length)
                 d_model, # the dimension of the initial linear layers for inputting patches into transformer
                 shared_recreation=True, # indicator of whether to project each channel individually or together
                 ):
        super().__init__()

        self.shared_recreation = shared_recreation
        self.n_vars = c_in

        # Input encoding: projection of feature vectors onto a d-dim vector space
        ## note that this could be an MLP too, if you want
        if not shared_recreation:
            self.W_P = nn.ModuleList()
            for _ in range(self.n_vars): self.W_P.append(nn.Linear(d_model, patch_len))
        else:
            self.W_P = nn.Linear(d_model, patch_len)

    def forward(self, x) -> Tensor:          
        """
        input: x: tensor [bs x nvars x d_model x num_patch]
        returns: x: tensor [bs x num_patch x nvars x d_model]
        """
        # Input embedding
        x = x.permute(0,3,1,2) # [bs x num_patch x nvars x d_model]
        if not self.shared_recreation:
            x_out = []
            for i in range(self.n_vars): 
                z = self.W_P[i](x[:,:,i,:])
                x_out.append(z)
            x = torch.stack(x_out, dim=2)
        else:
            x = self.W_P(x) # x: [bs x num_patch x nvars x patch_len]
        return x
    
class MaskedAutogressionFeedForward2(nn.Module):
    def __init__(self, d_model, patch_len, n_layers, dropout):
        super().__init__()
        self.n_layers = n_layers 
        self.layers = []
        n_nuerons = int((patch_len - d_model)/self.n_layers)
        if self.n_layers == 1:
            self.layers.append(nn.Sequential(nn.Linear(d_model, patch_len), nn.Dropout(dropout)))
        elif self.n_layers == 0:
            # do nothing
            self.layers.append(nn.Identity())
        else:
            for i in range(self.n_layers-1):
                self.layers.append(nn.Sequential(nn.Linear(n_nuerons*i+d_model, n_nuerons*(i+1)+d_model), nn.ReLU(), nn.Dropout(dropout)))
            self.layers.append(nn.Sequential(nn.Linear(n_nuerons*(i+1)+d_model, patch_len)))
        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        x = x.transpose(2,3)
        x = self.layers(x)
        x = x.permute(0,2,1,3)
        return x

## Miscellaneous

In [ ]:
#| export
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x, **kwargs):
        return x

In [ ]:
#| export
class Transpose(nn.Module):
    def __init__(self, *dims, contiguous=False): 
        super().__init__()
        self.dims, self.contiguous = dims, contiguous
    def forward(self, x):        
        if self.contiguous: return x.transpose(*self.dims).contiguous()
        else: return x.transpose(*self.dims)


def get_activation_fn(activation):
    if callable(activation): return activation()
    elif activation.lower() == "relu": return nn.ReLU()
    elif activation.lower() == "gelu": return nn.GELU()
    raise ValueError(f'{activation} is not available. You can use "relu", "gelu", or a callable')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()